In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Hyperparameters

num_epochs = 4
batch_size = 4
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', 
                                              train=True,
                                             download=True,
                                             transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', 
                                              train=False,
                                             download=True,
                                             transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)#3=input channel size, 6=output channel size, 5=kernel size
        self.pool = nn.MaxPool2d(2, 2)#kenrel size=2, stride=2
        self.conv2 = nn.Conv2d(6,16,5)#input channel size must be equal to output channel size (6), output channel size=16, kernel size =5
        self.fc1 = nn.Linear(16*5*5, 120)# 16*5*5 must be fixed. It flattens the image to one column.
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)# 10 ust be fixed as output of 10 classes
    def forward(self, x):
        # first convolutional and pooling layer
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        # second convolutional and pooling layer
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        # pass to FC layer
        x = x.view(-1, 16*5*5) # -1 gives us correct number of batches. Then the flattened size
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.fc3(x)
        
        return x
        

In [7]:
model = ConvNet()

criterion = nn.CrossEntropyLoss() # good for multiclass classification problems
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)

In [8]:

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step[{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            
print("Training complete.")

Epoch [1/4], Step[2000/12500], Loss: 2.2758
Epoch [1/4], Step[4000/12500], Loss: 2.3110
Epoch [1/4], Step[6000/12500], Loss: 2.3272
Epoch [1/4], Step[8000/12500], Loss: 2.2734
Epoch [1/4], Step[10000/12500], Loss: 2.1697
Epoch [1/4], Step[12000/12500], Loss: 2.4018
Epoch [2/4], Step[2000/12500], Loss: 2.1083
Epoch [2/4], Step[4000/12500], Loss: 1.7107
Epoch [2/4], Step[6000/12500], Loss: 1.4381
Epoch [2/4], Step[8000/12500], Loss: 2.0191
Epoch [2/4], Step[10000/12500], Loss: 1.4032
Epoch [2/4], Step[12000/12500], Loss: 1.6365
Epoch [3/4], Step[2000/12500], Loss: 1.6443
Epoch [3/4], Step[4000/12500], Loss: 1.2250
Epoch [3/4], Step[6000/12500], Loss: 1.5593
Epoch [3/4], Step[8000/12500], Loss: 0.9078
Epoch [3/4], Step[10000/12500], Loss: 1.7667
Epoch [3/4], Step[12000/12500], Loss: 1.2905
Epoch [4/4], Step[2000/12500], Loss: 1.3153
Epoch [4/4], Step[4000/12500], Loss: 1.1406
Epoch [4/4], Step[6000/12500], Loss: 1.2647
Epoch [4/4], Step[8000/12500], Loss: 1.0811
Epoch [4/4], Step[10000/12

In [9]:
with torch.no_grad():
    n_correct=0
    n_samples=0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            
            if (label == pred):
                n_class_correct[label] +=1
            
            n_class_samples[label] += 1
            
    acc = 100.0 * n_correct / n_samples
    
    print(f'Accuracy of the network {acc}%')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        
        print(f'Accuracy of {classes[i]}: {acc}%')
    
    

Accuracy of the network 46.49%
Accuracy of plane: 41.6%
Accuracy of car: 58.0%
Accuracy of bird: 27.6%
Accuracy of cat: 36.9%
Accuracy of deer: 17.4%
Accuracy of dog: 36.8%
Accuracy of frog: 58.2%
Accuracy of horse: 63.4%
Accuracy of ship: 63.4%
Accuracy of truck: 61.6%
